# Data Fit (Neural Networks): SIR data without noise

In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2025-01-15 13:43:38.844753: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-15 13:43:38.856689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 13:43:38.871949: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 13:43:38.876466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 13:43:38.888398: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1


I0000 00:00:1736959421.533701  614295 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736959421.583925  614295 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736959421.584194  614295 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


## Load data 

In [3]:
# Load Path
import pickle as pkl
data_path = '../../data/raw/simulated/SIR'
with open(f'{data_path}/sir.pkl', 'rb') as f:
    data_sim = pkl.load(f) 

## Division Training/Validation/Test

In [4]:

from sklearn.model_selection import train_test_split
import numpy as np

# Prepare data
X = [] # X will be the data from infected people
y = [] # y will be the parameters from that simulation

for simulation in data_sim:
    compartment_data = simulation[0]  # S, I, R matrix
    parameters = simulation[2]  # [beta, gamma]
    
    # Extract I values
    I_values = compartment_data[:, 1]  # second column columna (I compartment)
    
    X.append(I_values)  # Add to the characteristic list
    y.append(parameters)  # Add to label list

# Convert to numpy arrays
X = np.array(X) 
y = np.array(y) 

# First division: 90% training + validation, 10% test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Second división: 70% training, 20% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=2/9, random_state=42)

# Imprimir formas para verificar
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

X_train shape: (70000, 1001)
X_val shape: (20000, 1001)
X_test shape: (10000, 1001)
y_train shape: (70000, 2)
y_val shape: (20000, 2)
y_test shape: (10000, 2)


## Vanilla Neural Network Model

In [9]:
import tensorflow as tf

model = tf.keras.models.Sequential() 
model.add(tf.keras.layers.InputLayer(input_shape=[1001])) 
model.add(tf.keras.layers.Dense(120, activation='relu')) 
model.add(tf.keras.layers.Dense(240, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='linear')) 

from tensorflow.keras.callbacks import EarlyStopping

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics = [tf.keras.metrics.R2Score()])

early_stopping = EarlyStopping(patience = 5) # Número de iteraciones sin mejora de la pérdida de validación antes de parar el entrenamiento
history = model.fit(X_train, y_train,
                    validation_data = (X_val, y_val), 
                    epochs=100,
                    callbacks = [early_stopping])

Epoch 1/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 0.4877 - r2_score: -3.7531 - val_loss: 0.0040 - val_r2_score: 0.9669
Epoch 2/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0032 - r2_score: 0.9765 - val_loss: 0.0028 - val_r2_score: 0.9832
Epoch 3/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0039 - r2_score: 0.9737 - val_loss: 0.0015 - val_r2_score: 0.9904
Epoch 4/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0032 - r2_score: 0.9809 - val_loss: 0.0049 - val_r2_score: 0.9757
Epoch 5/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0030 - r2_score: 0.9814 - val_loss: 0.0018 - val_r2_score: 0.9891
Epoch 6/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0028 - r2_score: 0.9830 - val_loss: 0.0016 - val_r2_score: 0.9907
Epoch 7/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0022 - r2_score: 0.9861 - val_loss: 0.0013 - val_r2_score: 0.9911
Epoch 8/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0021 - r2_score:

In [10]:
model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 7.1557e-04 - r2_score: 0.9943


[0.0007062221993692219, 0.9944225549697876]

In [11]:
y_pred = model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [12]:
# Save Params fitted
with open(f'{data_path}/test/params_fit_NN.pkl', 'wb') as f:
    pkl.dump(y_pred, f)

In [13]:
y_test, y_pred

(array([[0.79131672, 0.52960316],
        [0.28326234, 0.16790258],
        [2.34309549, 0.84346774],
        ...,
        [1.84105089, 0.58375466],
        [2.97063045, 0.19425141],
        [1.00005171, 0.32324333]]),
 array([[0.8082745 , 0.5556596 ],
        [0.25960696, 0.2470391 ],
        [2.3231869 , 0.8352717 ],
        ...,
        [1.8470148 , 0.5899888 ],
        [2.968526  , 0.19090614],
        [1.0210208 , 0.33873   ]], dtype=float32))